# Wstęp

Metody uczenia maszynowego możemy podzielić na dwie główne kategorie (pomijając uczenie ze wzmocnieniem): nadzorowane i nienadzorowane. Uczenie **nadzorowane** (ang. *supervised*) to jest uczenie z dostępnymi etykietami dla danych wejściowych. Na parach danych uczących $dataset= \{(x_0,y_0), (x_1,y_1), \ldots, (x_n,y_n)\}$ model ma za zadanie nauczyć się funkcji $f: X \rightarrow Y$. Z kolei modele uczone w sposób **nienadzorowany** (ang. *unsupervised*) wykorzystują podczas trenowania dane nieetykietowane tzn. nie znamy $y$ z pary $(x, y)$.

Dość częstą sytuacją, z jaką mamy do czynienia, jest posiadanie małego podziobioru danych etykietowanych i dużego nieetykietowanych. Często annotacja danych wymaga ingerencji człowieka - ktoś musi określić co jest na obrazku, ktoś musi powiedzieć czy dane słowo jest rzeczownkiem czy czasownikiem itd.

Jeżeli mamy dane etykietowane do zadania uczenia nadzorowanego (np. klasyfikacja obrazka), ale także dużą ilość danych nieetykietowanych, to możemy wtedy zastosować techniki **uczenia częściowo nadzorowanego** (ang. *semi-supervised learning*). Te techniki najczęściej uczą się funkcji $f: X \rightarrow Y$, ale jednocześnie są w stanie wykorzystać informacje z danych nieetykietowanych do poprawienia działania modelu.

## Cel ćwiczenia

Celem ćwiczenia jest nauczenie modelu z wykorzystaniem danych etykietowanych i nieetykietowanych ze zbioru STL10 z użyciem metody [Bootstrap your own latent](https://arxiv.org/abs/2006.07733).

Metoda ta jest relatywnie "lekka" obliczeniowo, a także dość prosta do zrozumienia i zaimplementowania, dlatego też na niej się skupimy na tych laboratoriach.

# Zbiór STL10

Zbiór STL10 to zbiór stworzony i udostępniony przez Stanford [[strona]](https://ai.stanford.edu/~acoates/stl10/) [[papier]](https://cs.stanford.edu/~acoates/papers/coatesleeng_aistats_2011.pdf) a inspirowany przez CIFAR-10. Obrazy zostały pozyskane z [ImageNet](https://image-net.org/). Szczegóły można doczytać na ich stronie. To co jest ważne to to, że autorzy zbioru dostarczają predefiniowany plan eksperymentalny, żeby móc porównywać łatwo wyniki eksperymentów. Nie będziemy go tutaj stosować z uwagi na jego czasochłonność (10 foldów), ale warto pamiętać o tym, że często są z góry ustalone sposoby walidacji zaprojetowanych przez nas algorytmów na określonych zbiorach referencyjnych.

Korzystając z `torchvision.datasets` ***załaduj*** 3 podziały zbioru danych STL10: `train`, `test`, `unlabeled` oraz utwórz z nich instancje klasy `DataLoader`. Korzystając z Google Colab rozważ użycie Google Drive do przechowyania zbioru w calu zaoszczędzenia czasu na wielokrotne pobieranie.

In [2]:
import torchvision
from torch.utils.data import DataLoader

train_data = torchvision.datasets.STL10(
    root="./data",
    split="train",
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
test_data = torchvision.datasets.STL10(
    root="./data",
    split="test",
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
unlabelled_data = torchvision.datasets.STL10(
    root="./data",
    split="unlabeled",
    download=True,
    transform=torchvision.transforms.ToTensor(),
)


train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
unlabelled_loader = DataLoader(unlabelled_data, batch_size=64, shuffle=True)

100%|██████████| 2640397119/2640397119 [07:44<00:00, 5679391.68it/s] 


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [29]:
train_data.labels

array([1, 5, 1, ..., 1, 7, 5], dtype=uint8)

# Uczenie nadzorowane

Żeby porównać czy metoda BYOL przynosi nam jakieś korzyści musimy wyznaczyć wartość bazową metryk(i) jakości, których będziemu używać (np. dokładność).

***Zaimplementuj*** wybraną metodę uczenia nadzorowanego na danych `train` z STL10. Możesz wykorzystać predefiniowane architektury w `torchvision.models` oraz kody źródłowe z poprzednich list.

In [38]:
from torchvision.models import alexnet, AlexNet, AlexNet_Weights
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import classification_report
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the pre-trained model
model = alexnet(weights=AlexNet_Weights.DEFAULT)

# Modify the final layer to match the number of classes in STL-10 (10 classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train(
    model: AlexNet,
    train_loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    num_epochs: int = 10,
):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            # print(images)
            # print(labels)
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


def evaluate(model: AlexNet, test_loader: DataLoader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy}")


def evaluate_classification_report(model: AlexNet, test_loader: DataLoader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=test_data.classes))


train(model, train_loader, criterion, optimizer, num_epochs=1)

evaluate(model, test_loader)

KeyboardInterrupt: 

In [8]:
evaluate_classification_report(model, test_loader)

              precision    recall  f1-score   support

    airplane       0.10      1.00      0.18       800
        bird       0.00      0.00      0.00       800
         car       0.00      0.00      0.00       800
         cat       0.00      0.00      0.00       800
        deer       0.00      0.00      0.00       800
         dog       0.00      0.00      0.00       800
       horse       0.00      0.00      0.00       800
      monkey       0.00      0.00      0.00       800
        ship       0.00      0.00      0.00       800
       truck       0.00      0.00      0.00       800

    accuracy                           0.10      8000
   macro avg       0.01      0.10      0.02      8000
weighted avg       0.01      0.10      0.02      8000



/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [18]:
print(len(test_data))
print(test_loader.dataset.classes)
print(test_data[0][0].size())
print(test_data[0][1])

8000
['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
torch.Size([3, 96, 96])
6


# Bootstrap your own latent

Metoda [Bootstrap your own latent](https://arxiv.org/abs/2006.07733) jest opisana w rodziale 3.1 papieru a także w dodatku A. Składa się z dwóch etapów:


1.   uczenia samonadzorowanego (ang. *self-supervised*)
2.   douczania nadzorowanego (ang. *fine-tuning*)

## Uczenie samonadzorowane

Architektura do nauczania samonadzorowanego składa się z dwóch sieci: (1) *online* i (2) *target*. W uproszczeniu cała architektura działa tak:


1.   Dla obrazka $x$ wygeneruj dwie różne augmentacje $v$ i $v'$ za pomocą funkcji $t$ i $t'$.
2.   Widok $v$ przekazujemy do sieci *online*, a $v'$ do *target*.
3.   Następnie widoki przekształacamy za pomocą sieci do uczenia reprezentacji (np. resnet18 lub resnet50) do reprezentacji $y_\theta$ i $y'_\xi$.
4.   Potem dokonujemy projekcji tych reprezentacji w celu zmniejszenia wymiarowości (np. za pomocą sieci MLP).
5.   Na sieci online dokonujmey dodatkowo predykcji pseudo-etykiety (ang. *pseudolabel*)
6.   Wyliczamy fukncję kosztu: MSE z wyjścia predyktora sieci *online* oraz wyjścia projekcji sieci *target* "przepuszczonej" przez predyktor sieci *online* **bez propagacji wstecznej** (*vide Algorithm 1* z papieru).
7.   Dokonujemy wstecznej propagacji **tylko** po sieci *online*.
8.   Aktualizujemy wagi sieci *target* sumując w ważony sposób wagi obu sieci $\xi = \tau\xi + (1 - \tau)\theta$ ($\tau$ jest hiperprametrem) - jest to ruchoma średnia wykładnicza (ang. *moving exponential average*).

Po zakończeniu procesu uczenia samonadzorowanego zostawiamy do douczania sieć kodera *online* $f_\theta$. Cała sieć *target* oraz warstwy do projekcji i predykcji w sieci *online* są "do wyrzucenia".

### Augmentacja

Dodatek B publikacji opisuje augmentacje zastosowane w metodzie BYOL. Zwróć uwagę na tabelę 6 w publikacji. `torchvision.transforms.RandomApply` może być pomocne.

***Zaimeplementuj*** augmentację $\tau$ i $\tau'$.


In [56]:
import random
from torch import nn
import torch
from torchvision import transforms as T
from torchvision.transforms import functional as F

# Parameter T T′
# Random crop probability 1.0 1.0 x
# Flip probability 0.5 0.5 x
# Color jittering probability 0.8 0.8 x

# Brightness adjustment max intensity 0.4 0.4
# Contrast adjustment max intensity 0.4 0.4
# Saturation adjustment max intensity 0.2 0.2
# Hue adjustment max intensity 0.1 0.1

# Color dropping probability 0.2 0.2 x
# Gaussian blurring probability 1.0 0.1 x
# Solarization probability 0.0 0.2


def get_t1_aug():
    transform = T.Compose(
        [
            T.RandomResizedCrop(size=(96, 96)),
            T.RandomHorizontalFlip(p=0.5),
            RandomApply(
                T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1),
                p=0.8,
            ),
            RandomApply(T.Grayscale(num_output_channels=3), p=0.2),
            T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)),
        ]
    )
    return transform


def get_t2_aug():
    transform = T.Compose(
        [
            T.RandomResizedCrop(size=(94, 94)),
            T.RandomHorizontalFlip(p=0.5),
            RandomApply(
                T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1),
                p=0.8,
            ),
            RandomApply(T.Grayscale(num_output_channels=3), p=0.2),
            RandomApply(T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)), p=0.1),
            RandomApply(F.solarize, p=0.2),
        ]
    )
    return transform


class RandomApply(nn.Module):
    def __init__(self, fn: nn.Module, p: float):
        super().__init__()
        self.fn = fn
        self.p = p

    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)

### Implementacja uczenia samonadzorowanego

***Zaprogramuj*** proces uczenia samonadzorowanego na danych nieetykietowanych ze zbioru STL10.

Wskazówki do realizacji polecenia:

1. Proces uczenia może trwać bardzo długo dlatego zaleca się zastsowanie wczesnego zatrzymania lub uczenia przez tylko jedną epokę. Mimo wszystko powinno się dać osiągnąć poprawę w uczeniu nadzorowanym wykorzystując tylko zasoby z Google Colab.
2. Dobrze jest pominąć walidację na zbiorze treningowym i robić ją tylko na zbiorze walidacyjnym - zbiór treningowy jest ogromny i w związku z tym narzut czasowy na walidację też będzie duży.
3. Walidację modelu można przeprowadzić na zbiorze `train` lub całkowicie ją pominąć, jeżeli uczymy na stałej ilości epok.
4. Rozważ zastosowanie tylko jednej augmentacji - augmentacja $\tau'$ jest bardziej czasochłonna niż $\tau$.
5. Poniżej jest zaprezentowany zalążek kodu - jest on jedynie wskazówką i można na swój sposób zaimplementować tę metodę

In [65]:
from copy import deepcopy
from json import encoder
from torch import le, nn
from torch import nn, Tensor
from torch.nn import functional as F

from src.ssl_base import SSLBase
import copy


class SmallConvnet(nn.Module):
    """Small ConvNet (avoids heavy computation)."""

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 21 * 21, 120)
        self.fc2 = nn.Linear(120, 84)

    def forward(self, x: Tensor) -> Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 16 * 21 * 21)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x


class MLP(nn.Module):
    def __init__(
        self, input_dim: int, hidden_dim: int, output_dim: int, plain_last: bool = False
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, output_dim),
        )
        if not plain_last:
            self.net.append(nn.BatchNorm1d(output_dim))
            self.net.append(nn.ReLU(inplace=True))

    def forward(self, x: Tensor) -> Tensor:
        return self.net(x)


def mlp(dim: int, projection_size: int = 256) -> nn.Module:
    return nn.Sequential(
        nn.Linear(dim, projection_size),
        nn.ReLU(),
        nn.BatchNorm1d(projection_size),
        nn.ReLU(),
    )


class BYOLModel(SSLBase):
    def __init__(
        self,
        learning_rate: float,
        weight_decay: float,
        tau: float,
        out_channels: int = 10,
    ):
        super().__init__(
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            out_channels=out_channels,
        )

        # Initialize online network
        # funkcja f
        self.online_encoder = SmallConvnet()

        # funkcja g
        self.online_projector = MLP(84, 84, 84, plain_last=False)

        # funkcja q
        self.online_predictor = MLP(84, 84, 84, plain_last=True)
        self.online_net = nn.Sequential(
            self.online_encoder,
            self.online_projector,
            self.online_predictor,
        )

        # Initialize target network with frozen weights
        self.target_encoder = self.copy_and_freeze_module(self.online_encoder)
        self.target_projector = self.copy_and_freeze_module(self.online_projector)
        self.target_net = nn.Sequential(self.target_encoder, self.target_projector)

        # Initialize augmentations
        self.aug_1 = get_t1_aug()
        self.aug_2 = get_t1_aug()

        self.tau = tau

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        t = self.aug_1(x)
        t_prim = self.aug_2(x)

        q = self.online_net(t)
        q_sym = self.online_net(t_prim)

        with torch.no_grad():
            z_prim = self.target_net(t_prim)
            z_prim_sym = self.target_net(t)

        q = torch.cat([q, q_sym], dim=0)
        z_prim = torch.cat([z_prim, z_prim_sym], dim=0)

        return q, z_prim

    def forward_repr(self, x: Tensor) -> Tensor:
        return self.online_encoder(x)

    def training_step(self, batch: Tensor, batch_idx: int) -> Tensor:
        x, _ = batch
        q, z_prim = self.forward(x)
        loss = self.byol_loss(q=q, z_prim=z_prim)

        self.log("train/loss", loss, prog_bar=True)

        return loss

    def byol_loss(self, q: Tensor, z_prim: Tensor) -> Tensor:
        q = F.normalize(q, dim=-1, p=2)
        z_prim = F.normalize(z_prim, dim=-1, p=2)
        return (2 - 2 * (q * z_prim).sum(dim=-1)).mean()

    def on_train_epoch_end(self) -> None:
        super().on_train_epoch_end()
        self.update_target_network()

    @torch.no_grad()
    def update_target_network(self) -> None:
        for target_param, online_param in zip(
            self.target_net.parameters(), self.online_net.parameters()
        ):
            target_param.data = (
                self.tau * target_param.data + (1 - self.tau) * online_param.data
            )

    @staticmethod
    def copy_and_freeze_module(model: nn.Module) -> nn.Module:
        mode_copy = copy.deepcopy(model)
        for param in mode_copy.parameters():
            param.requires_grad = False

        return mode_copy


aug_1 = get_t1_aug()
aug_2 = get_t1_aug()

model = BYOLModel(
    learning_rate=1e-3,
    weight_decay=1e-5,
    tau=0.99,
    out_channels=10,
)

In [66]:
from cgitb import small
from lightning import Trainer

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
TAU = 0.99
EPOCHS = 200
ACCELERATOR = "cuda" if torch.cuda.is_available() else "cpu"
OUT_DIR = "logs"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# logger = TensorBoardLogger(save_dir=OUT_DIR, default_hp_metric=False)

x = next(iter(train_loader))[0].size()


trainer = Trainer(
    default_root_dir=OUT_DIR,
    max_epochs=EPOCHS,
    # logger=logger,
    accelerator=ACCELERATOR,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
)

trainer.fit(model, train_loader)
trainer.test(model, test_loader)


def test_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=test_data.classes))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name             | Type         | Params
--------------------------------------------------
0 | online_encoder   | SmallConvnet | 859 K 
1 | online_projector | MLP          | 14.6 K
2 | online_predictor | MLP          | 14.4 K
3 | online_net       | Sequential   | 888 K 
4 | target_encoder   | SmallConvnet | 859 K 
5 | target_projector | MLP          | 14.6 K
6 | target_net       | Sequential   | 874 K 
--------------------------------------------------
888 K     Trainable params
874 K     Non-trainable params
1.8 M     Total params
7.054     Total estimated model params size (MB)
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-pac

Epoch 5:  44%|████▍     | 35/79 [00:18<00:22,  1.93it/s, v_num=12, train/loss=0.561]

/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/piotr/projects/ai/gsn-l/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'train_dataloader'

## Douczanie nadzorowane

***Zaimplementuj*** proces douczania kodera z poprzedniego polecenia na danych etykietowanych ze zbioru treningowego. Porównaj jakość tego modelu z modelem nauczonym tylko na danych etykietownaych. Postaraj się wyjaśnić różnice.

In [68]:
state_dict = model.online_encoder.state_dict()
encoder = SmallConvnet()
encoder.load_state_dict(state_dict)


class SupervisedModel(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.fc = nn.Linear(84, 10)

    def forward(self, x):
        x = self.encoder(x)
        x = self.fc(x)
        return x


supervised_model = SupervisedModel(encoder)
supervised_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised_model.parameters(), lr=0.001)

train(supervised_model, train_loader, criterion, optimizer, num_epochs=10)
evaluate(supervised_model, test_loader)
evaluate_classification_report(supervised_model, test_loader)

Epoch 0:   0%|          | 0/79 [55:34<?, ?it/s]


KeyboardInterrupt: 